Write an pyspark code to find the ctr of each Ad.Round ctr to 2 
decimal points. Order the result table by ctr in descending order 
and by ad_id in ascending order in case of a tie. 
Ctr=Clicked/(Clicked+Viewed)

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.sql.functions import *
from pyspark.sql.window import Window


data = [ 
(1, 1, 'Clicked'), 
(2, 2, 'Clicked'), 
(3, 3, 'Viewed'), 
(5, 5, 'Ignored'), 
(1, 7, 'Ignored'), 
(2, 7, 'Viewed'), 
(3, 5, 'Clicked'), 
(1, 4, 'Viewed'), 
(2, 11, 'Viewed'), 
(1, 2, 'Clicked') 
] 

schema = StructType([
  StructField("AD_ID", IntegerType(), True),
  StructField("USER_ID", IntegerType(), True),
  StructField("ACTION", StringType(), True)
])

df_click = spark.createDataFrame(data, schema)
df_click.display()

#Ctr=Clicked/(Clicked+Viewed)
df_click_grp = df_click.groupBy("AD_ID").agg(sum(when(col("ACTION") == "Clicked", 1)\
  .otherwise(0)).alias("clicked"),sum(when(col("ACTION") == "Viewed", 1)\
  .otherwise(0)).alias("Viewed"))

df_clk_ctr = df_click_grp.withColumn("CTR", round(try_divide(col("clicked"), col("Viewed")+ col('clicked')),2))

df_clk_ctr.selectExpr("AD_ID", "CTR as CTR").orderBy(col("CTR").desc()).display()

In [0]:
# The try_divide function safely divides two columns, returning null if the denominator is zero or null, instead of raising an error.
# Example:
# try_divide(numerator, denominator)
# If denominator is zero or null, result is null; otherwise, result is numerator/denominator.

Combine Two DF 
Write a Pyspark program to report the first name, last name, city, and state of each person in the 
Person dataframe. If the address of a personId is not present in the Address dataframe, 
report null instead.

In [0]:
# Define schema for the 'persons' table 
persons_schema = StructType([ 
StructField("personId", IntegerType(), True), 
StructField("lastName", StringType(), True), 
StructField("firstName", StringType(), True) 
]) 
# Define schema for the 'addresses' table 
addresses_schema = StructType([ 
StructField("addressId", IntegerType(), True), 
StructField("personId", IntegerType(), True), 
StructField("city", StringType(), True), 
StructField("state", StringType(), True) 
]) 
# Define data for the 'persons' table 
persons_data = [ 
(1, 'Wang', 'Allen'), 
(2, 'Alice', 'Bob') 
] 
# Define data for the 'addresses' table 
addresses_data = [ 
(1, 2, 'New York City', 'New York'), 
(2, 3, 'Leetcode', 'California') 
]

df_person = spark.createDataFrame(persons_data, persons_schema)
df_address = spark.createDataFrame(addresses_data, addresses_schema)
df_person.display()
df_address.display()

df_join = df_person.join(df_address, df_person.personId == df_address.personId, "left")\
    .select("firstName","lastName","city","state")
       
df_join.display()

04_Employees Earning More Than Their Managers 
Write a Pyspark program to find Employees Earning More Than Their 
Managers


In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.sql.functions import *
from pyspark.sql.window import Window

# Define the schema for the "employees" 
employees_schema = StructType([ 
StructField("id", IntegerType(), True), 
StructField("name", StringType(), True), 
StructField("salary", IntegerType(), True), 
StructField("managerId", IntegerType(), True) 
]) 
# Define data for the "employees" 
employees_data = [ 
(1, 'Joe', 70000, 3), 
(2, 'Henry', 80000, 4), 
(3, 'Sam', 60000, None), 
(4, 'Max', 90000, None) 
] 

df_emp = spark.createDataFrame(employees_data, employees_schema)
df_emp.display()

df_empl = df_emp.alias("emp")
df_mgr = df_emp.alias("mgr")


ddf_emp_mgr = df_empl.join(df_mgr, df_empl.managerId == df_mgr.id, "left")
emp_sal = ddf_emp_mgr.filter(df_empl['salary'] > df_mgr['salary'])
emp_sal.display()

Duplicate Emails 
Write a Pyspark program to report all the duplicate emails. 
Note that it's guaranteed that the email field is not NULL. 

In [0]:
schema = StructType([ 
StructField("id", IntegerType(), True), 
StructField("email", StringType(), True)
])  

# Define data for the "employees" 
data = [ 
(1, 'a@b.com'), 
(2, 'c@d.com'), 
(3, 'a@b.com')
] 

df_eml = spark.createDataFrame(data, schema)
df_eml.display()

df_email = df_eml.groupBy(col('email')).agg(count('email').alias("cnt"))
df_fileml = df_email.filter(col('cnt') > 1).select("email")
df_fileml.display()

06_Customers Who Never Order - 
Write a Pyspark program to find all customers who never 
order anything.

In [0]:
# Define the schema for the "Customers" 
customers_schema = StructType([ 
StructField("id", IntegerType(), True), 
StructField("name", StringType(), True) 
]) 

# Define data for the "Customers" 
customers_data = [ 
(1, 'Joe'), 
(2, 'Henry'), 
(3, 'Sam'), 
(4, 'Max') 
] 

# Define the schema for the "Orders" 
orders_schema = StructType([ 
StructField("id", IntegerType(), True), 
StructField("customerId", IntegerType(), True) 
]) 

# Define data for the "Orders" 
orders_data = [ 
(1, 3), 
(2, 1) 
]

df_cust = spark.createDataFrame(customers_data, customers_schema)
df_ord = spark.createDataFrame(orders_data, orders_schema)
df_cust.display()
df_ord.display()

df_noorder = df_cust.join(df_ord, df_ord.customerId == df_cust.id, "leftanti")
df_noorder.display()


07_Rising Temperature - 

Write a solution to find all dates' Id with higher 
temperatures compared to its previous dates (yesterday). 
Return the result table in any order.

In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

# Define the schema for the "Weather" table 
weather_schema = StructType([ 
StructField("id", IntegerType(), True), 
StructField("recordDate", StringType(), True), 
StructField("temperature", IntegerType(), True) 
]) 


# Define data for the "Weather" table 
weather_data = [ 
(1, '2015-01-01', 10), 
(2, '2015-01-02', 25), 
(3, '2015-01-03', 20), 
(4, '2015-01-04', 30)
]

df_temp = spark.createDataFrame(weather_data, weather_schema)
df_temp.display()

from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window

win_spec= Window.orderBy("id")
prev_date = df_temp.withColumn("prev_temp", lag(col("temperature")).over(win_spec))\
  .filter(col('temperature') > col('prev_temp'))\
  .select("id")
  
prev_date.display()

08_Game Play Analysis I 

Write a solution to find the first login date for each player. 
Return the result table in any order.

In [0]:
# Define the schema for the "Activity" 
activity_schema = StructType([ 
StructField("player_id", IntegerType(), True), 
StructField("device_id", IntegerType(), True), 
StructField("event_date", StringType(), True), 
StructField("games_played", IntegerType(), True) 
]) 
 
# Define data for the "Activity" 
activity_data = [ 
(1, 2, '2016-03-01', 5), 
(1, 2, '2016-05-02', 6), 
(2, 3, '2017-06-25', 1), 
(3, 1, '2016-03-02', 0), 
(3, 4, '2018-07-03', 5) 
]

df_login = spark.createDataFrame(activity_data, activity_schema)
df_login.display()

df_date = df_login.withColumn("event_date", to_date(col("event_date"),'yyyy-MM-dd'))
df_date.display()

win_spec = Window.partitionBy(col("player_id")).orderBy("event_date")
log_dt = df_date.withColumn('rank',  dense_rank().over(win_spec)).filter(col('rank') == 1)
log_dt.display()

log_dt.selectExpr("player_id", "event_date as first_login").display()

09_Game Play Analysis II 

Write a pyspark code that reports the device that is first 
logged in for each player. 
Return the result table in any order.

In [0]:
# Define the schema for the "Activity" 
activity_schema = StructType([ 
StructField("player_id", IntegerType(), True), 
StructField("device_id", IntegerType(), True), 
StructField("event_date", StringType(), True), 
StructField("games_played", IntegerType(), True) 
]) 

# Define data for the "Activity" 
activity_data = [ 
(1, 2, '2016-03-01', 5), 
(1, 2, '2016-05-02', 6), 
(2, 3, '2017-06-25', 1), 
(3, 1, '2016-03-02', 0), 
(3, 4, '2018-07-03', 5) 
]

df_login = spark.createDataFrame(activity_data, activity_schema)
df_login.display()

df_date = df_login.withColumn("event_date", to_date(col("event_date"),'yyyy-MM-dd'))
df_date.display()

win_spec = Window.partitionBy(col("player_id")).orderBy("event_date")
log_dt = df_date.withColumn('rank',  dense_rank().over(win_spec)).filter(col('rank') == 1)
log_dt.display()

log_dt.selectExpr("player_id", "device_id as first_login").display()

10_Employee Bonus 

Write a solution to report the name and bonus amount of 
each employee with a bonus less than 1000. 
Return the result table in any order

In [0]:
# Define the schema for the "Employee" 
employee_schema = StructType([ 
StructField("empId", IntegerType(), True), 
StructField("name", StringType(), True), 
StructField("supervisor", IntegerType(), True), 
StructField("salary", IntegerType(), True) 
]) 

# Define data for the "Employee" 
employee_data = [ 
(3, 'Brad', None, 4000), 
(1, 'John', 3, 1000), 
(2, 'Dan', 3, 2000), 
(4, 'Thomas', 3, 4000) 
] 

# Define the schema for the "Bonus" 
bonus_schema = StructType([ 
StructField("empId", IntegerType(), True), 
StructField("bonus", IntegerType(), True) 
]) 

# Define data for the "Bonus" 
bonus_data = [ 
(2, 500), 
(4, 2000) 
] 

df_emp = spark.createDataFrame(employee_data, employee_schema)
df_bonus = spark.createDataFrame(bonus_data, bonus_schema)
df_emp.display()
df_bonus.display()

emp_alias = df_emp.alias('e')
df_bonus_alias = df_bonus.alias('b')


df_emp_bon = emp_alias.join(df_bonus_alias, (col("e.empId") == col("b.empId")), "inner")\
  .filter(col('bonus') < 1000)

emp_sel = df_emp_bon.select(col("e.empId"), col("e.name"), col("b.bonus"))

emp_sel.display()

11_Find Customer Referee 

Find the names of the customer that are not referred by the 
customer with id = 2. 
Return the result table in any order

In [0]:
# Define the schema for the Customer table 
schema = StructType([ 
StructField("id", IntegerType(), True), 
StructField("name", StringType(), True), 
StructField("referee_id", IntegerType(), True) 
]) 

# Create an RDD with the data 
data = [ 
(1, 'Will', None), 
(2, 'Jane', None), 
(3, 'Alex', 2), 
(4, 'Bill', None), 
(5, 'Zack', 1), 
(6, 'Mark', 2) 
]
 
df_ref = spark.createDataFrame(data, schema)
df_ref.display()

df_not_ref = df_ref.filter((col("referee_id") != 2) | col("referee_id").isNull()).select("name")

df_not_ref.display()

12_Cities With Completed Trades 

Write a pypsark code to retrieve the top three cities that 
have the highest number of completed trade orders listed in 
descending order. Output the city name and the 
corresponding number of completed trade orders.

In [0]:
# Define the schema for the trades 
trades_schema = StructType([ 
StructField("order_id", IntegerType(), True), 
StructField("user_id", IntegerType(), True), 
StructField("price", FloatType(), True), 
StructField("quantity", IntegerType(), True), 
StructField("status", StringType(), True), 
StructField("timestamp", StringType(), True) 
]) 

# Define the schema for the users 
users_schema = StructType([ 
StructField("user_id", IntegerType(), True), 
StructField("city", StringType(), True), 
StructField("email", StringType(), True), 
StructField("signup_date", StringType(), True) 
]) 

# Create an RDD with the data for trades 
trades_data = [ 
(100101, 111, 9.80, 10, 'Cancelled', '2022-08-17 12:00:00'), 
(100102, 111, 10.00, 10, 'Completed', '2022-08-17 12:00:00'), 
(100259, 148, 5.10, 35, 'Completed', '2022-08-25 12:00:00'), 
(100264, 148, 4.80, 40, 'Completed', '2022-08-26 12:00:00'), 
(100305, 300, 10.00, 15, 'Completed', '2022-09-05 12:00:00'), 
(100400, 178, 9.90, 15, 'Completed', '2022-09-09 12:00:00'), 
(100565, 265, 25.60, 5, 'Completed', '2022-12-19 12:00:00') 
] 

# Create an RDD with the data for users 
users_data = [ 
(111, 'San Francisco', 'rrok10@gmail.com', '2021-08-03 12:00:00'), 
(148, 'Boston', 'sailor9820@gmail.com', '2021-08-20 12:00:00'), 
(178, 'San Francisco', 'harrypotterfan182@gmail.com', '2022-01-05 12:00:00'), 
(265, 'Denver', 'shadower_@hotmail.com', '2022-02-26 12:00:00'), 
(300, 'San Francisco', 'houstoncowboy1122@hotmail.com', '2022-06-30 12:00:00') 
] 

df_trades = spark.createDataFrame(trades_data, trades_schema)
df_users = spark.createDataFrame(users_data, users_schema)
df_trades.display()
df_users.display()

df_td_use_join = df_trades.alias('t').join(df_users.alias('u'), (col("t.user_id") == col("u.user_id")),"inner")
df_td_use_join.display()


df_filt = df_td_use_join.filter(col("status") == 'Completed')

df_city = df_filt.groupBy("u.city").agg(count("city").alias("count"))
  

df_city.display()


13_Page With No Likes 

Write a pyspark code to return the IDs of the Facebook pages 
that have zero likes. The output should be sorted in 
ascending order based on the page IDs. 

In [0]:
# Define the schema for the pages 
pages_schema = StructType([ 
StructField("page_id", IntegerType(), True), 
StructField("page_name", StringType(), True) 
]) 

# Define the schema for the page_likes table 
page_likes_schema = StructType([ 
StructField("user_id", IntegerType(), True), 
StructField("page_id", IntegerType(), True), 
StructField("liked_date", StringType(), True) 
]) 

# Create an RDD with the data for pages 
pages_data = [ 
(20001, 'SQL Solutions'), 
(20045, 'Brain Exercises'), 
(20701, 'Tips for Data Analysts') 
] 

# Create an RDD with the data for page_likes table 
page_likes_data = [ 
(111, 20001, '2022-04-08 00:00:00'), 
(121, 20045, '2022-03-12 00:00:00'), 
(156, 20001, '2022-07-25 00:00:00') 
]

df_pages = spark.createDataFrame(pages_data, pages_schema)
df_page_likes = spark.createDataFrame(page_likes_data, page_likes_schema)
df_pages.display()
df_page_likes.display()

df_page_join = df_pages.alias('p').join(df_page_likes.alias('l'), (col("p.page_id") == col("l.page_id")), "leftanti")
df_page_join.display()

14_Purchasing Activity by Product 

Type 
We have been given purchasing activity DF and we need 
to find out cumulative purchases of each product over 
time. 

In [0]:

# Define schema for the DataFrame 
schema = StructType([ 
StructField("order_id", IntegerType(), True), 
StructField("product_type", StringType(), True), 
StructField("quantity", IntegerType(), True), 
StructField("order_date", StringType(), True), 
]) 
 
# Define data 
data = [ 
(213824, 'printer', 20, "2022-06-27 "), 
(212312, 'hair dryer', 5, "2022-06-28 "), 
(132842, 'printer', 18, "2022-06-28 "),   
(284730, 'standing lamp', 8, "2022-07-05 ") 
] 

df_cum = spark.createDataFrame(data, schema)
df_cum.display()

# df_pur_dt = df_cum.groupBy(col("order_id")).agg(sum("quantity").alias("total_quantity"))\
#   .selectExpr("order_date", "product_type", "total_quantity as cum_purchased")
# df_pur_dt.display()


win_spec = Window.partitionBy("product_type").orderBy("order_date").rowsBetween(Window.unboundedPreceding, 0) 

# Add a new column 'cumulative_purchases' representing the cumulative sum 

result_df = df_cum.withColumn("cumulative_purchases", sum("quantity").over(win_spec)) 
result_df.display()


**15_Teams Power Users 

Write a pyspark code to identify the top 2 Power Users 
who sent the highest number of messages on Microsoft 
Teams in August 2022. Display the IDs of these 2 users 
along with the total number of messages they sent. 
Output the results in descending order based on the 
count of the messages.**

In [0]:
schema = StructType([ 
StructField("message_id", IntegerType(), True), 
StructField("sender_id", IntegerType(), True), 
StructField("receiver_id", IntegerType(), True), 
StructField("content", StringType(), True), 
StructField("sent_date", StringType(), True), 
]) 

# Define the data 
data = [ 
(901, 3601, 4500, 'You up?', '2022-08-03 00:00:00'), 
(902, 4500, 3601, 'Only if you\'re buying', '2022-08-03 00:00:00'), 
(743, 3601, 8752, 'Let\'s take this offline', '2022-06-14 00:00:00'), 
(922, 3601, 4500, 'Get on the call', '2022-08-10 00:00:00'), 
]

df_msg = spark.createDataFrame(data, schema)
df_msg.display()

df_msg_date= df_msg.filter(col("sent_date").like("2022-08%"))

df_msg_cnt =df_msg_date.groupBy("sender_id").agg(count("message_id").alias("cnt"))

df_msg_cnt.display()

16_Select in pyspark 

Write a pyspark code perform below function 
● Write a pyspark code to get all employee detail. 
● Write a query to get only "FirstName" column from emp_df 
● Write a Pyspark code to get FirstName in upper case as "First 
Name". 
● Write a pyspark code to get FirstName in lower case

17_Select in pyspark 
Write a pyspark code perform below function

● Write a pyspark code for combine FirstName and LastName 
and display it as "Name" (also include white space between 
first name & last name) 
● Select employee detail whose name is "Vikas" 
● Get all employee detail from EmployeeDetail table whose 
"FirstName" start with letter 'a'.

18_Select in pyspark 

Write a pyspark code perform below function 
● Get all employee details from EmployeeDetail table whose 
"FirstName" contains 'k' 
●  Get all employee details from EmployeeDetail table whose 
"FirstName" end with 'h' 
● Get all employee detail from EmployeeDetail table whose 
"FirstName" start with 
● Get all employee detail from EmployeeDetail table whose 
"FirstName" start with any single character between 'a-p'

19_Select in pyspark 

Write a pyspark code perform below function 
●  Get all employee detail from emp_df whose "Gender" end 
with 'le' and contain 4 letters. The Underscore(_) Wildcard 
Character represents any single character. 
●  Get all employee detail from EmployeeDetail table whose 
"FirstName" start with # 'A' and contain 5 letters. 
● Get all unique "Department" from EmployeeDetail table. 
● Get the highest "Salary" from EmployeeDetail table.

20_Date in pyspark 

Write a pyspark code perform below function 
● Get the lowest "Salary" from EmployeeDetail table. 
● Show "JoiningDate" in "dd mmm yyyy" format, ex- "15 Feb 
2013" 
● Show "JoiningDate" in "yyyy/mm/dd" format, ex- "2013/02/15" 

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType
from pyspark.sql.functions import *
from pyspark.sql.window import Window


data = [ 
[1, "Vikas", "Ahlawat", 600000.0, "2013-02-15 11:16:28.290", "IT", 
"Male"], 
[2, "nikita", "Jain", 530000.0, "2014-01-09 17:31:07.793", "HR", "Female"], 
[3, "Ashish", "Kumar", 1000000.0, "2014-01-09 10:05:07.793", "IT", 
"Male"], 
[4, "Nikhil", "Sharma", 480000.0, "2014-01-09 09:00:07.793", "HR", 
"Male"], 
[5, "anish", "kadian", 500000.0, "2014-01-09 09:31:07.793", "Payroll", 
"Male"], 
] 

# Create a schema for the DataFrame 
schema = StructType([ 
StructField("EmployeeID", IntegerType(), True), 
StructField("First_Name", StringType(), True), 
StructField("Last_Name", StringType(), True), 
StructField("Salary", DoubleType(), True), 
StructField("Joining_Date", StringType(), True), 
StructField("Department", StringType(), True), 
StructField("Gender", StringType(), True) 
])

df_emp = spark.createDataFrame(data, schema)
df_emp.display()


def emp_details(df, columns):
    columns = df.columns
    df = df.withColumn(df.columns[1], upper(df.columns[1]))\
        .withColumn(df.columns[2], lower(df.columns[2]))\
        .withColumn("full_name", concat_ws(' ', col(df.columns[1]), col(df.columns[2])))\
        .select(columns + ['full_name'])
    return df   
   
emp_df = emp_details(df_emp, df_emp.columns)
emp_df.display()


#17 - filter based on names

def name_filter(df):
    columns = df.columns
    df = df.filter((col("First_Name") == "Vikas") | (col("Last_name").startswith("J")))
    return df

df_names = name_filter(df_emp)
display(df_names)


#18 - filter based on name pattern

def name_pattern(df):
    columns = df.columns
    df = df.filter(col("First_Name").contains("k") | (col("First_Name").endswith('h') | (col("First_name").rlike(r"^[a-p]"))))
    return df

n_pattern = name_pattern(df_emp)
display(n_pattern)

# 19 - filter name based on 'contains' characters

def name_contains(df):
    columns = df.columns
    df = df.filter(((col("Gender").endswith("le")) & (length(col("Gender")) == 4)) | (col("First_Name").startswith("A") & (length(col("First_Name")) == 5)))

    return df     

df_cont = name_contains(df_emp)
display(df_cont)

# df1 = df_emp.groupby(col("Department")).agg(max(col("Salary")).alias("max_salary"))
# df1.display()

# 20 - DATE FUNCTIONS

def emp_lowsal(df):
    df = df.agg(max(col("Salary")).alias("max_salary"), min(col("Salary")).alias("Lowsalary"))
    return df

min_max_sal = emp_lowsal(df_emp)
display(min_max_sal)


# def date_filter(df, columns, dateformat = ["yyyy-MM-dd HH:mm:ss.SSS"]):
#     columns = df.columns
#     for col in df.columns:
#         if col == "Joining_Date":
#             df = df.withColumn("Joining_Date_new", date_format(to_timestamp(col("Joining_Date"),"yyyy-MM-dd HH:mm:ss.SSS"),'dd mmm yyyy'))
#         return df

# df_date = date_filter(df_emp, df_emp.columns)
# display(df_date)

df_date_fmt = df_emp.withColumn("Joining_Date_fmt", date_format(to_timestamp(col("Joining_Date"), "yyyy-MM-dd HH:mm:ss.SSS"), "dd MMM yyyy"))\
        .withColumn("join_fmt_new", to_date(to_timestamp(col("Joining_Date"), "yyyy-MM-dd HH:mm:ss.SSS"), "yyyy-MM-dd"))
display(df_date_fmt)


In [0]:
df_date_fmt = df_emp.withColumn(
    "Joining_Date_fmt",
    date_format(to_timestamp(col("Joining_Date"), "yyyy-MM-dd HH:mm:ss.SSS"), "dd MMM yyyy"))\
        .withColumn("join_fmt_new", to_date(to_timestamp(col("Joining_Date"), "yyyy-MM-dd HH:mm:ss.SSS"), "yyyy-MM-dd"))

display(df_date_fmt)

21_Date in pyspark 

Write a pyspark code perform below function 
● Get only Year part of "JoiningDate" 
● Get only Month part of "JoiningDate". 
● Get only date part of "JoiningDate". 
● Get the current system date using DataFrame API 
● Get the current UTC date and time using DataFrame API

In [0]:
def date_part_extract(df):
    df = df.withColumn("year", year(col("Joining_Date")))\
        .withColumn("Month", month(col("Joining_Date")))\
        .withColumn("Day", dayofmonth(col("Joining_Date")))\
        .withColumn("current_date", current_date())\
        .withColumn("current_timestamp", current_timestamp())
    return df

df_date_part = date_part_extract(df_date_fmt)
display(df_date_part)   

22_Date in pyspark 

Write a pyspark code perform below function 
● Get the first name, current date, joiningdate and diff between 
current date and joining date in months. 
● Get the first name, current date, joiningdate and diff between 
current date and joining date in days. 
● Get all employee details from EmployeeDetail table whose 
joining year is 2013

In [0]:
data = [ 
[1, "Vikas", "Ahlawat", 600000.0, "2013-02-15 11:16:28.290", "IT", "Male"], 
[2, "nikita", "Jain", 530000.0, "2014-01-09 17:31:07.793", "HR", "Female"], 
[3, "Ashish", "Kumar", 1000000.0, "2014-01-09 10:05:07.793", "IT", "Male"], 
[4, "Nikhil", "Sharma", 480000.0, "2014-01-09 09:00:07.793", "HR", "Male"], 
[5, "anish", "kadian", 500000.0, "2014-01-09 09:31:07.793", "Payroll", "Male"], 
] 

# Create a schema for the DataFrame 
schema = StructType([ 
StructField("EmployeeID", IntegerType(), True), 
StructField("First_Name", StringType(), True), 
StructField("Last_Name", StringType(), True), 
StructField("Salary", DoubleType(), True), 
StructField("Joining_Date", StringType(), True), 
StructField("Department", StringType(), True), 
StructField("Gender", StringType(), True) 
])

df_dt = spark.createDataFrame(data, schema)
df_dt.display()

def date_dif(df):
  df = df_emp.filter(year(to_timestamp(col("Joining_Date"), "yyyy-MM-dd HH:mm:ss.SSS")) == 2013).selectExpr(
                     "First_Name", 
                     "current_date() as current_date", 
                     "Joining_Date", 
                     "round(months_between(current_date(), to_timestamp(Joining_Date, 'yyyy-MM-dd HH:mm:ss.SSS')),2) as months_diff",
                     "date_diff(current_date(), to_timestamp(Joining_Date, 'yyyy-MM-dd HH:mm:ss.SSS')) as days_diff")            
      
  return df

df_date_diff = date_dif(df_dt)
display(df_date_diff)


# from pyspark.sql.functions import year, to_timestamp, col

# df_filtered_year = df_emp.filter(year("Joining_Date")== 2013)

# display(df_filtered_year)


In [0]:
from pyspark.sql.functions import current_date, months_between, col, to_timestamp

def date_dif(df):
    df = df.withColumn(
        "current_date", current_date()
    ).withColumn(
        "Joining_Date_ts", to_timestamp(col("Joining_Date"), "yyyy-MM-dd HH:mm:ss.SSS")
    ).withColumn(
        "months_diff", months_between(col("current_date"), col("Joining_Date_ts"))
    ).select(
        "First_Name", "current_date", "Joining_Date", "months_diff"
    )
    return df

df_date_diff = date_dif(df_dt)
display(df_date_diff)

23_Date in pyspark 

Write a pyspark code perform below function 
● Get all employee details from EmployeeDetail table whose 
joining month is Jan(1). 
● Get all employee details from EmployeeDetail table whose 
joining date between 2013-01-01" and "2013-12-01". 
● Get how many employee exist in "EmployeeDetail" table. 
● Select all employee detail with First name "Vikas","Ashish", and 
"Nikhil".

In [0]:
data = [ 
[1, "Vikas", "Ahlawat", 600000.0, "2013-02-15 11:16:28.290", "IT", "Male"], 
[2, "nikita", "Jain", 530000.0, "2014-01-09 17:31:07.793", "HR", "Female"], 
[3, "Ashish", "Kumar", 1000000.0, "2014-01-09 10:05:07.793", "IT", "Male"], 
[4, "Nikhil", "Sharma", 480000.0, "2014-01-09 09:00:07.793", "HR", "Male"], 
[5, "anish", "kadian", 500000.0, "2014-01-09 09:31:07.793", "Payroll", "Male"], 
] 

# Create a schema for the DataFrame 
schema = StructType([ 
StructField("EmployeeID", IntegerType(), True), 
StructField("First_Name", StringType(), True), 
StructField("Last_Name", StringType(), True), 
StructField("Salary", DoubleType(), True), 
StructField("Joining_Date", StringType(), True), 
StructField("Department", StringType(), True), 
StructField("Gender", StringType(), True) 
]) 

df_date_det = spark.createDataFrame(data, schema)
df_date_det.display()

from pyspark.sql.functions import current_date, months_between, col, months

def date_det(df):
    columns = df.columns
    df = df.filter((month("Joining_Date") == 1))\
        .select(*columns)
    return df

df_date_det = date_det(df_date_det)
display(df_date_det)

def date_det1(df):
    columns = df.columns
    df = df.filter(col("Joining_Date").between("2013-01-01", "2013-12-01"))\
        .select(*columns)
    return df

df_date = date_det1(df_emp)
display(df_date)

df_cnt = df_emp.agg(count("EmployeeID").alias("empcnt"))
display(df_cnt)

df_fname =df_emp.filter(col("First_Name").isin("Vikas","Ashish","Nikhil"))
df_fname.display()


24_Trim and case in pyspark 

Write a pyspark code perform below function 
● Select all employee detail with First name not in 
"Vikas","Ashish", and "Nikhil". 
● Select first name from "EmployeeDetail" df after removing 
white spaces from right side 
● Select first name from "EmployeeDetail" table after removing 
white spaces from left side 
● Display first name and Gender as M/F.(if male then M, if 
Female then F) 

In [0]:
data = [ 
[1, "Vikas", "Ahlawat", 600000.0, "2013-02-15 11:16:28.290", "IT", "Male"], 
[2, "nikita", "Jain", 530000.0, "2014-01-09 17:31:07.793", "HR", "Female"], 
[3, "Ashish", "Kumar", 1000000.0, "2014-01-09 10:05:07.793", "IT", "Male"], 
[4, "Nikhil", "Sharma", 480000.0, "2014-01-09 09:00:07.793", "HR", "Male"], 
[5, "anish", "kadian", 500000.0, "2014-01-09 09:31:07.793", "Payroll", "Male"], 
] 

# Create a schema for the DataFrame 
schema = StructType([ 
StructField("EmployeeID", IntegerType(), True), 
StructField("First_Name", StringType(), True), 
StructField("Last_Name", StringType(), True), 
StructField("Salary", DoubleType(), True), 
StructField("Joining_Date", StringType(), True), 
StructField("Department", StringType(), True), 
StructField("Gender", StringType(), True)
])

df_not_fname = spark.createDataFrame(data, schema)
df_not_fname.display()

def not_fname(df):
    columns = df.columns
    df = df.filter(~df["First_Name"].isin("Vikas","Ashish", "Nikhil"))\
        .withColumn("Gender", when(df["Gender"] == 'Male', 'M').otherwise('F'))\
        .select(rtrim(ltrim("First_Name")).alias("First_Name"), "Joining_Date", "Department", "Gender")
    return df

not_fname_df= not_fname(df_not_fname)
not_fname_df.display()


25_operator in pyspark 

Write a pyspark code perform below function 
● Select first name from "EmployeeDetail" table prifixed with 
"Hello " 
● Get employee details from "EmployeeDetail" table whose 
Salary greater than 600000 
● Get employee details from "EmployeeDetail" table whose 
Salary less than 700000 
● Get employee details from "EmployeeDetail" table whose 
Salary between 500000 than 600000 
Difficult Level : EASY 
DataFrame: 
● Select second highest salary from "EmployeeDetail" table 

In [0]:


from pyspark.sql.functions import col, concat, lit

def emp_sal(df):
    df_greater = df.withColumn("First_Name", concat(lit("Hello"), col("First_Name")))\
                .filter(col("Salary") >= 600000)     
    df_less = df.filter(col("Salary") < 700000)
    df_between = df.filter(col("Salary").between(500000, 600000))   
    df_sec_sal = df.withColumn("sec_sal", row_number().over(Window.orderBy(desc("Salary"))))\
                .filter(col("sec_sal") == 2) # (or) orderBy(col("Salary").desc()).limit(2).offset(1)

    return df_greater, df_less, df_between, df_sec_sal

df_greater, df_less, df_between, df_sec_sal = emp_sal(df_emp)

display(df_greater)
display(df_less)
display(df_between)
display(df_sec_sal)

 # df_sec_highsal = df.orderBy(col("Salary").desc()).limit(1)


26_groupby in pyspark 
Write a pyspark code perform below function 

●  Write the query to get the department and department wise 
  total(sum) salary from "EmployeeDetail" table. 
●  Write the query to get the department and department wise 
total(sum) salary, display it in ascending order according to 
salary. 
● Write the query to get the department and department wise 
total(sum) salary, display it in descending order according to 
salary. 
●  Write the query to get the department, total no. of 
departments, total(sum) salary with respect to department 
from "EmployeeDetail" table.

In [0]:
# Creating Spark Session 
from pyspark.sql import SparkSession 
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType
from pyspark.sql.functions import col, lit, sum, count, row_number, desc
from pyspark.sql.window import Window


data = [ 
 [1, "Vikas", "Ahlawat", 600000.0, "2013-02-15 11:16:28.290", "IT", "Male"], 
 [2, "nikita", "Jain", 530000.0, "2014-01-09 17:31:07.793", "HR", "Female"], 
 [3, "Ashish", "Kumar", 1000000.0, "2014-01-09 10:05:07.793", "IT", "Male"], 
 [4, "Nikhil", "Sharma", 480000.0, "2014-01-09 09:00:07.793", "HR", "Male"], 
 [5, "anish", "kadian", 500000.0, "2014-01-09 09:31:07.793", "Payroll", "Male"], 
]

# Create a schema for the DataFrame 
schema = StructType([ 
 StructField("EmployeeID", IntegerType(), True), 
 StructField("First_Name", StringType(), True), 
 StructField("Last_Name", StringType(), True), 
 StructField("Salary", DoubleType(), True), 
 StructField("Joining_Date", StringType(), True), 
 StructField("Department", StringType(), True), 
 StructField("Gender", StringType(), True) 
]) 
 
df_detemp = spark.createDataFrame(data, schema)
df_detemp.display()


def emp_det(df):
    df_sal_tot = df.groupBy('Department').agg((sum('Salary')).alias('Sum_Of_salary'))
    df_asc = df_sal_tot.orderBy(col("Sum_Of_salary").asc())
    df_desc = df_sal_tot.orderBy(col("Sum_Of_salary").desc())
    df_dept = df.groupBy("Department").agg(count("Department").alias("No_of_dept"),(sum("Salary")).alias("Total_Salary"))
    return df_sal_tot, df_asc, df_desc, df_dept

# df_name_var = [df_sal_tot, df_asc, df_desc, df_dept]

df_sal_tot, df_asc, df_desc, df_dept = emp_det(df_detemp)
df_sal_tot.display()
df_asc.display()
df_desc.display()
df_dept.display()



# for i in df_name_var:   
#     i.display()

# display(df_sal, df_asc, df_desc, df_dept)

#display all the dataframe dynamically without defining the function
# import sys

# def display_all_dataframes():
#     for df_name, df_val in globals().items():
#         if 'DataFrame' in str(type(df_val)):
#             display(df_val)

# display_all_dataframes() 



27_groupby in pyspark 
Write a pyspark code perform below function 

●  46. Get department wise average salary from 
"EmployeeDetail" table order by salary ascending 
●  47. Get department wise maximum salary from 
"EmployeeDetail" table order by salary ascending 
● 48. Get department wise minimum salary from 
"EmployeeDetail" table order by salary ascending

In [0]:
from pyspark.sql.functions import avg, min, max

data = [ 
 [1, "Vikas", "Ahlawat", 600000.0, "2013-02-15 11:16:28.290", "IT", "Male"], 
 [2, "nikita", "Jain", 530000.0, "2014-01-09 17:31:07.793", "HR", "Female"], 
 [3, "Ashish", "Kumar", 1000000.0, "2014-01-09 10:05:07.793", "IT", "Male"], 
 [4, "Nikhil", "Sharma", 480000.0, "2014-01-09 09:00:07.793", "HR", "Male"], 
 [5, "anish", "kadian", 500000.0, "2014-01-09 09:31:07.793", "Payroll", "Male"], 
]  
 
# Create a schema for the DataFrame 
schema = StructType([ 
 StructField("EmployeeID", IntegerType(), True), 
 StructField("First_Name", StringType(), True), 
 StructField("Last_Name", StringType(), True), 
 StructField("Salary", DoubleType(), True), 
 StructField("Joining_Date", StringType(), True), 
 StructField("Department", StringType(), True), 
 StructField("Gender", StringType(), True) 
])

df_dept_sal = spark.createDataFrame(data, schema)
df_dept_sal.display()

def dept_salary(df):
    df_sal_avg = df.groupBy("Department").agg(avg('Salary').alias("avg_saalry")).orderBy(col("avg_saalry").asc())
    df_sal_min = df.groupBy("Department").agg(min('Salary').alias("min_salary")).orderBy(col("min_salary").asc())
    df_sal_max = df.groupBy("Department").agg(max('Salary').alias("max_salary")).orderBy(col("max_salary").asc())
    return df_sal_avg, df_sal_min, df_sal_max

df_sal_avg, df_sal_min, df_sal_max = dept_salary(df_dept_sal)

df_sal_avg.display()
df_sal_min.display()
df_sal_max.display()



# for i in df_name:
#     i.display()

# display(df_sal_avg, df_sal_min, df_sal_max)


# not working for the individual cell to run all the dataframe

# def display_add_df():
#     for df_name, df_val in globals().items():
#         if 'DataFrame' in str(type(df_val)):
#             display(df_val)

# display_add_df()



28_Join_in_pyspark 

Write a pyspark code perform below function 
●  Write down the query to fetch Project name assign to more 
than one Employee 
● Get employee name, project name order by firstname from 
"EmployeeDetail" and"ProjectDetail" for those employee which 
have assigned project already.

In [0]:
data = [ 
 [1, "Vikas", "Ahlawat", 600000.0, "2013-02-15 11:16:28.290", "IT", "Male"], 
 [2, "nikita", "Jain", 530000.0, "2014-01-09 17:31:07.793", "HR", "Female"], 
 [3, "Ashish", "Kumar", 1000000.0, "2014-01-09 10:05:07.793", "IT", "Male"], 
 [4, "Nikhil", "Sharma", 480000.0, "2014-01-09 09:00:07.793", "HR", "Male"], 
 [5, "anish", "kadian", 500000.0, "2014-01-09 09:31:07.793", "Payroll", "Male"]
]  

# Create a schema for the DataFrame 
schema = StructType([ 
 StructField("EmployeeID", IntegerType(), True), 
 StructField("First_Name", StringType(), True), 
 StructField("Last_Name", StringType(), True), 
 StructField("Salary", DoubleType(), True), 
 StructField("Joining_Date", StringType(), True), 
 StructField("Department", StringType(), True), 
 StructField("Gender", StringType(), True) 
]) 
 
df_e = spark.createDataFrame(data, schema)
df_e.display()
 
pro_schema = StructType([ 
 StructField("Project_DetailID", IntegerType(), True), 
 StructField("Employee_DetailID", IntegerType(), True), 
 StructField("Project_Name", StringType(), True) 
]) 
  
# Create the data as a list of tuples 
pro_data = [ 
 (1, 1, "Task Track"), 
 (2, 1, "CLP"), 
 (3, 1, "Survey Management"), 
 (4, 2, "HR Management"), 
 (5, 3, "Task Track"), 
 (6, 3, "GRS"), 
 (7, 3, "DDS"), 
 (8, 4, "HR Management"), 
 (9, 6, "GL Management")]

pro_df = spark.createDataFrame(pro_data, pro_schema)
pro_df.display()

def proj_details(df, df1):
    emp_proj_join = df.alias('e').join(df1.alias('p'), (col("e.EmployeeID") == col("p.Employee_DetailID")), "inner")
    proj_more = emp_proj_join.groupBy('Project_Name').agg(count("Project_DetailID").alias("count_proj")).filter(col("count_proj") > 1)

    proj_emp_name = emp_proj_join.select("First_Name", "Project_Name").orderBy(col('First_Name').asc())

    return emp_proj_join, proj_more,proj_emp_name


emp_proj_join, proj_more, proj_emp_name = proj_details(df_e, pro_df)
emp_proj_join.display()
proj_more.display()
proj_emp_name.display()


29_Join_in_pyspark 
Write a pyspark code perform below function 

●  52. Get employee name, project name order by firstname from 
"EmployeeDetail" and "ProjectDetail" for all employee even 
they have not assigned project. 

● 53  Get employee name, project name order by firstname from 
"EmployeeDetail" and "ProjectDetail" for all employee if 
project is not assigned then display "-No Project Assigned".

In [0]:
data = [ 
 [1, "Vikas", "Ahlawat", 600000.0, "2013-02-15 11:16:28.290", "IT", "Male"], 
 [2, "nikita", "Jain", 530000.0, "2014-01-09 17:31:07.793", "HR", "Female"], 
 [3, "Ashish", "Kumar", 1000000.0, "2014-01-09 10:05:07.793", "IT", "Male"], 
 [4, "Nikhil", "Sharma", 480000.0, "2014-01-09 09:00:07.793", "HR", "Male"], 
 [5, "anish", "kadian", 500000.0, "2014-01-09 09:31:07.793", "Payroll", "Male"]
]  

# Create a schema for the DataFrame 
schema = StructType([ 
 StructField("EmployeeID", IntegerType(), True), 
 StructField("First_Name", StringType(), True), 
 StructField("Last_Name", StringType(), True), 
 StructField("Salary", DoubleType(), True), 
 StructField("Joining_Date", StringType(), True), 
 StructField("Department", StringType(), True), 
 StructField("Gender", StringType(), True) 
]) 
 
df_emp_noproj = spark.createDataFrame(data, schema)
df_emp_noproj.display()
 
pro_schema = StructType([ 
 StructField("Project_DetailID", IntegerType(), True), 
 StructField("Employee_DetailID", IntegerType(), True), 
 StructField("Project_Name", StringType(), True) 
]) 
  
# Create the data as a list of tuples 
pro_data = [ 
 (1, 1, "Task Track"), 
 (2, 1, "CLP"), 
 (3, 1, "Survey Management"), 
 (4, 2, "HR Management"), 
 (5, 3, "Task Track"), 
 (6, 3, "GRS"), 
 (7, 3, "DDS"), 
 (8, 4, "HR Management"), 
 (9, 6, "GL Management")]

pro_noemp_df = spark.createDataFrame(pro_data, pro_schema)
pro_noemp_df.display()


def No_proj_assign(df,df1):
    no_proj_emp = df.alias('e').join(df1.alias('p'), (col("e.EmployeeID") == col("p.Employee_DetailID")), "left").select("EmployeeID","First_Name","Last_Name","Project_Name").orderBy(col("First_Name").asc())

    if no_proj_emp.filter(col("Project_Name").isNull()):
        no_proj_null = no_proj_emp.fillna("No Project Assigned", subset=["Project_Name"])

        return no_proj_emp, no_proj_null

no_proj_emp, no_proj_null = No_proj_assign(df_emp_noproj,pro_noemp_df)
no_proj_emp.display()
no_proj_null.display()


30_Join_in_pyspark 
Write a pyspark code perform below function 

●  56. Write a pyspark code to find out the employeename who 
has not assigned any project, and display "-No Project Assigned"( tables :- [EmployeeDetail], [ProjectDetail]). 

● 57. Write a pyspark code to find out the project name which is 
not assigned to any employee( tables :- [EmployeeDetail],[ProjectDetail]).

In [0]:
from pyspark.sql.functions import col, lit, when
from pyspark.sql.types import *
from pyspark.sql.functions import *

data = [ 
 [1, "Vikas", "Ahlawat", 600000.0, "2013-02-15 11:16:28.290", "IT", "Male"], 
 [2, "nikita", "Jain", 530000.0, "2014-01-09 17:31:07.793", "HR", "Female"], 
 [3, "Ashish", "Kumar", 1000000.0, "2014-01-09 10:05:07.793", "IT", "Male"], 
 [4, "Nikhil", "Sharma", 480000.0, "2014-01-09 09:00:07.793", "HR", "Male"], 
 [5, "anish", "kadian", 500000.0, "2014-01-09 09:31:07.793", "Payroll", "Male"], 
] 

 
# Create a schema for the DataFrame 
schema = StructType([ 
 StructField("EmployeeID", IntegerType(), True), 
 StructField("First_Name", StringType(), True), 
 StructField("Last_Name", StringType(), True), 
 StructField("Salary", DoubleType(), True), 
 StructField("Joining_Date", StringType(), True), 
 StructField("Department", StringType(), True), 
 StructField("Gender", StringType(), True) 
]) 
 
pro_schema = StructType([ 
 StructField("Project_DetailID", IntegerType(), True), 
 StructField("Employee_DetailID", IntegerType(), True), 
 StructField("Project_Name", StringType(), True) 
]) 
  
# Create the data as a list of tuples 
pro_data = [ 
 (1, 1, "Task Track"), 
 (2, 1, "CLP"), 
 (3, 1, "Survey Management"), 
 (4, 2, "HR Management"), 
 (5, 3, "Task Track"), 
 (6, 3, "GRS"), 
 (7, 3, "DDS"), 
 (8, 4, "HR Management"), 
 (9, 6, "GL Management") 
] 

df_emp = spark.createDataFrame(data, schema)
pro_df = spark.createDataFrame(pro_data, pro_schema)

def no_project(df, df1): 
    df_emp_proj = df.alias('e').join(df1.alias('p'), (col("e.EmployeeID") == (col('p.Employee_DetailID'))), 'left')
    df_emp_proj_null = df_emp_proj.filter(col("Project_Name").isNull()).withColumn('Project_Name', when(col("Project_Name").isNull(), "No Project Assigned").otherwise(col("Project_Name"))).select("First_Name", "Project_Name")

    df_proj_not_assign = df.alias('e').join(df1.alias('p'), (col("e.EmployeeID") == (col('p.Employee_DetailID'))), 'right')
    df_emp_null = df_proj_not_assign.filter(col("First_Name").isNull()).select("Project_Name")
                                            
    return df_emp_proj, df_emp_proj_null, df_emp_null

 
df_emp_proj, df_emp_proj_null, df_emp_null = no_project(df_emp, pro_df)

df_emp_proj.display()
df_emp_proj_null.display()
df_emp_null.display()

31_Histogram of Tweets 

write a query to obtain a histogram of tweets posted per user in 2022. Output the 
tweet count per user as the bucket and the number of Twitter users who fall into that 
bucket. 

In other words, group the users by the number of tweets they posted in 2022 and 
count the number of users in each group.

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

schema = StructType([ 
 StructField("tweet_id", IntegerType(), True), 
 StructField("user_id", IntegerType(), True), 
 StructField("msg", StringType(), True), 
 StructField("tweet_date", StringType(), True) 
]) 
  
# Define the data 
data = [ 
 (214252, 111, 'Am considering taking Tesla private at $420. Funding secured.', '2021-12-30 00:00:00'), 
 (739252, 111, 'Despite the constant negative press covfefe', '2022-01-01 00:00:00'), 
 (846402, 111, 'Following @NickSinghTech on Twitter changed my life!', '2022-02-14 00:00:00'), 
 (241425, 254, 'If the salary is so competitive why won’t you tell me what it is?', '2022-03-01 00:00:00'), 
 (231574, 148, 'I no longer have a manager. I can\'t be managed', '2022-03 23 00:00:00') 
]

df_tweet = spark.createDataFrame(data, schema)
df_tweet.display()

def get_user(df):
    df_user = df.groupBy("user_id").agg(count("tweet_id").alias("tweet_count"))

    df_year = df.withColumn("year", year(expr("try_cast(tweet_date as timestamp)")))\
        .filter(col("year") == 2022)\
        .groupBy("user_id")\
        .agg(count("tweet_id").alias("tweet_count"))

    df_ucnt =df_year.groupBy('tweet_count').agg(count('user_id').alias("user_cnt")).select("tweet_count", "user_cnt")
    
    return df_user, df_year, df_ucnt

d_user = [d_user,df_year,df_ucnt]
d_user = get_user(df_tweet)

for i in d_user:
    i.display()



32_pyspark_transformation 

Write a pyspark code to transform the DataFrame to display each student's marks in Math and English as 
separate columns. 

In [0]:
data=[ 
('Rudra','math',79), 
('Rudra','eng',60), 
('Shivu','math', 68), 
('Shivu','eng', 59), 
('Anu','math', 65), 
('Anu','eng',80) 
] 
  
schema = StructType([ 
 StructField("Name", StringType(), True), 
 StructField("Sub", StringType(), True), 
 StructField("Marks", IntegerType(), True) 
])

df_marks = spark.createDataFrame(data, schema)
df_marks.display()

df_pivot = df_marks.groupBy("Name").pivot("Sub").agg(sum("Marks"))
df_pivot.display()

# df_sql = df_marks.withColumn("pivot", expr('''
#                    select * from (
#                        pivot(
#                        sum(Marks) for sub in ('math','eng')
#                    ))'''))

# df_sql.display()

# df_s = df_marks.createOrReplaceTempView("marks")


# df_st = spark.sql('''SELECT *
# FROM (
#     SELECT Name,
#            Sub        
#     FROM marks
# )
# PIVOT (
#     SUM(Marks)
#     FOR sub IN ('math','eng')
# )''')
# df_st.display()
 


# df_marks.createOrReplaceTempView("marks")

# df_sql = spark.sql("""
#     SELECT
#         Name,
#         SUM(CASE WHEN Sub = 'math' THEN Marks ELSE 0 END) AS math,
#         SUM(CASE WHEN Sub = 'eng' THEN Marks ELSE 0 END) AS eng
#     FROM marks
#     GROUP BY Name
# """)
# df_sql.display()

33_Hobbies Data Transformation 

 Problem Statement: 
Transform a dataset with individuals' names and 
associated hobbies into a new format using PySpark. 
Convert the comma-separated hobbies into separate 
rows, creating a DataFrame with individual rows  for 
each person and their respective hobbies. 

In [0]:
# Sample input data 
data = [("Alice", "badminton,tennis"), 
        ("Bob", "tennis,cricket"), 
        ("Julie", "cricket,carroms")] 
  
# Create a DataFrame 
df_hobby = spark.createDataFrame(data, ["name", "hobbies"])
df_hobby.display()

df_hob_split = df_hobby.withColumn("hobbies", explode(split(col("hobbies"), ",")))
df_hob_split.display()

34_ Histogram of Tweets 

 write a pyspark code to obtain a histogram of tweets 
posted per user in 2022. Output the tweet count per 
user as the bucket and the number of Twitter users who 
fall into that bucket.In other words, group the users by 
the number of tweets they posted in 2022 and count the 
number of users in each group.

In [0]:
schema = StructType([ 
 StructField("tweet_id", IntegerType(), True), 
 StructField("user_id", IntegerType(), True), 
 StructField("msg", StringType(), True), 
 StructField("tweet_date", StringType(), True) 
]) 
  
# Define the data 
data = [ 
 (214252, 111, 'Am considering taking Tesla private at $420. Funding secured.', '2021-12-30 00:00:00'), 
 (739252, 111, 'Despite the constant negative press covfefe', '2022-01-01 00:00:00'), 
 (846402, 111, 'Following @NickSinghTech on Twitter changed my life!', '2022-02-14 00:00:00'), 
 (241425, 254, 'If the salary is so competitive why won’t you tell me what it is?', '2022-03-01 00:00:00'), 
 (231574, 148, 'I no longer have a manager. I can\'t be managed', '2022-03 23 00:00:00') 
]

df_tweet = spark.createDataFrame(data, schema)
df_tweet.display()

def get_user(df):
    df_user = df.groupBy("user_id").agg(count("tweet_id").alias("tweet_count"))

    df_year = df.withColumn("year", year(expr("try_cast(tweet_date as timestamp)")))\
        .filter(col("year") == 2022)\
        .groupBy("user_id")\
        .agg(count("msg").alias("cnt"))

    df_ucnt =df_year.groupBy(col('cnt')).agg(count('user_id').alias("bucket")).select(col("cnt"),"bucket")
    
    return df_user, df_year, df_ucnt

d_user = [d_user,df_year,df_ucnt]
d_user = get_user(df_tweet)

for i in d_user:
    i.display()

35_Classes More Than 5 Students 

 Write a pyspark code to find all the classes that have at 
least five students.Return the result table in any order.

In [0]:
# Define the schema for the DataFrame 
schema = StructType([ 
 StructField("StudentID", StringType(), True), 
 StructField("ClassName", StringType(), True) 
]) 

# Data to be inserted into the DataFrame 
data = [ 
 ('A', 'Math'), 
 ('B', 'English'), 
 ('C', 'Math'), 
 ('D', 'Biology'), 
 ('E', 'Math'), 
 ('F', 'Computer'), 
 ('G', 'Math'), 
 ('H', 'Math'), 
 ('I', 'Math') 
]

df_students = spark.createDataFrame(data, schema)
df_students.display() 

df_stud_cnt = df_students.groupBy("ClassName").agg(count("StudentID").alias("stud_cnt"))\
  .filter(col("stud_cnt") > 5)
df_stud_cnt.display()

36_Rank Scores Problem 

 Write a pyspark code to rank scores. If there is a tie between 
two scores, both should have the same ranking. Note that 
after a tie, the next ranking number should be the next 
consecutive integer value.In other words, there should be no 
“holes” between ranks.

In [0]:
from pyspark.sql.functions import col, when, dense_rank, spark_partition_id
from pyspark.sql.types import *
from pyspark.sql.window import Window

# Define the schema for the DataFrame 
schema = StructType([ 
 StructField("Id", IntegerType(), True), 
 StructField("Score", DoubleType(), True) 
]) 
  
# Data to be inserted into the DataFrame 
data = [ 
 (1, 3.50), 
 (2, 3.65), 
 (3, 4.00), 
 (4, 3.85), 
 (5, 4.00), 
 (6, 3.65) 
] 

df_rank = spark.createDataFrame(data, schema)
df_rank.display()

def score(df):
  df_r = df.withColumn("Dense_Rank", dense_rank().over(Window.orderBy(col("Score")))).withColumn("Partition_id", spark_partition_id())
  return df_r

df_score = score(df_rank)
df_score.display()


37_Triangle Judgement Problem 
  
A pupil Tim gets homework to identify whether three line segments could 
possibly form a triangle. 

However, this assignment is very heavy because there are hundreds of records 
to calculate.  

Could you help Tim by writing a pyspark code to judge whether these three 
sides can form a triangle,assuming df triangle holds the length of the three sides x, y and z.

In [0]:
schema = StructType([ 
 StructField("x", IntegerType(), True), 
 StructField("y", IntegerType(), True), 
 StructField("z", IntegerType(), True) 
]) 
  
# Data to be inserted into the DataFrame 
data = [ 
 (13, 15, 30), 
 (10, 20, 15)]

df_triangle = spark.createDataFrame(data, schema)
df_triangle.display()

df_tri = df_triangle.withColumn("valid_triangle", when((col("x") +col("y") > col("z")) & (col("x") + col("z") > col("y")) & (col("x") + col("y") > col("z")), "Yes").otherwise("No"))
df_tri.display() 


38_Biggest Single Number Problem 
 
Df contains many numbers in column num including duplicated ones. 
Can you write a pyspark code to find the biggest number, which only 
appears once. 

In [0]:
# Define the schema for the DataFrame 
schema = StructType([StructField("num", IntegerType(), True)]) 
  
# Your data 
data = [(8,), (8,), (3,), (3,), (1,), (4,), (5,), (6,)] 
  
# Create a PySpark DataFrame 
df_dup = spark.createDataFrame(data, schema=schema)
df_dup.display()

df_duplicate = df_dup.withColumn("Row_Number", row_number().over(Window.partitionBy(col("num")).orderBy(col("num").desc()))).filter(col("Row_Number") > 1).drop("Row_Number")
df_duplicate.display()

df_djoin = df_dup.join(df_duplicate, on="num", how="leftanti")
df_max = df_djoin.agg(max(col("num")).alias("Max_Num"))
df_djoin.display()
df_max.display()

# df_d = list(df_dup)
# print(df_d)
# df_nd = list(df_duplicate)
# print(df_nd)

# non_dup =[]
# for i in df_d:
#     if df_d != df_nd:
#         non_dup.append(i)
#         print(non_dup)
   

39_Not Boring Movies Problem 

 X city opened a new cinema, many people would like to go to this 
cinema.   The cinema also gives out a poster indicating the movies’ ratings 
and descriptions.  Please write a Pyspark Code to output movies with an 
odd numbered ID and  a description that is not ‘boring’. Order the result 
by rating.

In [0]:
# Define the schema for the DataFrame 
schema = StructType([ 
 StructField("id", IntegerType(), True), 
 StructField("movie", StringType(), True), 
 StructField("description", StringType(), True), 
 StructField("rating", FloatType(), True) 
]) 
  
# Your data 
data = [ 
 (1, "War", "great 3D", 8.9), 
 (2, "Science", "fiction", 8.5), 
 (3, "Irish", "boring", 6.2), 
 (4, "Ice song", "Fantasy", 8.6), 
 (5, "House card", "Interesting", 9.1) 
]

df_movie = spark.createDataFrame(data, schema)
df_movie.display()

# df_movie1 = df_movie.withColumn("rating", col("rating").cast("double"))
# df_movie1.display()

df_nbore = df_movie.filter(~(col("description") == "boring") & (col("id") % 2 == 1))
df_nbore.display()

40_Swap Gender Problem 

   Given a df salary, such as the one below, that has m=male 
and f=female values. Swap all f and m values (i.e., change all f 
values to m and vice versa) 

In [0]:
# Define the schema for the DataFrame 
schema = StructType([ 
 StructField("id", IntegerType(), True), 
 StructField("name", StringType(), True), 
 StructField("sex", StringType(), True), 
 StructField("salary", IntegerType(), True), 
]) 
  
# Define the data 
data = [ 
 (1, "A", "m", 2500), 
 (2, "B", "f", 1500), 
 (3, "C", "m", 5500), 
 (4, "D", "f", 500), 
] 

df_gender = spark.createDataFrame(data, schema)
df_gender.display() 

df_gender_swap = df_gender.withColumn("sex", when(col("sex") == "m", "f").otherwise("m"))
df_gender_swap.display()

df_col = df_gender.columns

def swapgen(df):
    df_s = lambda x:x.replace('m','f')
    df_s = lambda x:x.replace('f','m')
    return df_s

df_gen = swapgen(df_gender.columns[2])
print(df_gen)


    # if 'sex' in df_col:
    #     df_s = lambda x:x.replace('m','f')
    # else:
    #     df_s = lambda x:x.replace('f','m')     

 


ddf_gender_swapped = df_gender.replace(
    {"m": "f", "f": "m"},
    subset=["sex"]
)

display(ddf_gender_swapped)